In [1]:
# Installing the dependencies needed for the project

In [2]:
# tenserflow, opencv mediapipe sklearn and matplotlib

In [3]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1

  Using cached https://files.pythonhosted.org/packages/ad/fc/fccaa149d7ccc165de01d62d19e5e9492e87ad23a7106f6dfe132800ca6f/tensorflow-2.4.1-cp38-cp38-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/e3/3e/f545952fbd966bf3864cf1565c2f68f9e645a18f9c32f9cb6ac5e28e3698/tensorflow_gpu-2.4.1-cp38-cp38-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/1e/63/0d32c1803c08518dd03e02f3cfe302335624f511155be723bcc7329fed4e/h5py-2.10.0-cp38-cp38-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/d9/41/be8f5e0acc98a134ac149f570d953b98a65ecf25989e63ba90fd6b7cff4b/grpcio-1.32.0-cp38-cp38-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/79/4c/7c3275a01e12ef9368a892926ab932b33bb13d55794881e3573482b378a7/Keras_Preprocessing-1.1.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f7/fd/67c61276de025801cfa8a1b9af2d7c577e7f27c17b6bff2baca20bf03543/tensorboard-2.8.0-py3-none-any.whl
  Using cached https://fil

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\anthc\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [5]:
pip install mediapipe

  Using cached fonttools-4.31.1-py3-none-any.whl (899 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\anthc\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\anthc\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [7]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\anthc\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [8]:
# Importing dependendencies

In [9]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp